In [1]:
import time
import os

import pandas as pd
import numpy as np

import requests
from selenium import webdriver
from bs4 import BeautifulSoup

In [2]:
#os.chdir('/c/Users/matth/Documents/Coding/RA Fouirnaies/vs_ltm_linktable/')

In [3]:
#!pwd

# Vote Smart - FollowTheMoney Link Table

## Overview

This notebook documents the process of creating a link table between the enterprise/organization ID of Vote Smart and FollowTheMoney. We begin with a list of the desired organizations' names as seen on the Vote Smart website and scrape the FTM website search queries to obtain unique IDs. 

### Search Query Exploration

#### Example: Campaign for Working Families

Here we seek to understand the format of the search query output and create a method to extract the desired information from each query while handling/providing discussion of how to handle exceptions.

In [4]:
url = 'https://www.followthemoney.org/search-results/SearchForm?Search=Campaign+for+Working+Families'
headers = {'Access-Control-Allow-Origin': '*',
          'Access-Control-Allow-Methods': 'GET',
          'Access-Control-Allow-Headers': 'Content-Type',
          'Access-Control-Max-Age': '3600'}

**Issue**

Direct parsing of the html using beautifulsoup does not work since the table produced by the search query is a dynamic table, and as a result you need to actually make a request to the website for it to show up.

Debugging for dynamic table issue

https://stackoverflow.com/questions/17597424/how-to-retrieve-the-values-of-dynamic-html-content-using-python
https://stackoverflow.com/questions/40208051/selenium-using-python-geckodriver-executable-needs-to-be-in-path
https://www.howtogeek.com/118594/how-to-edit-your-system-path-for-easy-command-line-access/amp/

**Workaround**

Use selenium webdriver (requires firefox and geckodriver) to access the table. This is not ideal since it requires certain downloads/specific file paths in addition to opening the firefox broswer. Based on what I have read, it appears that this is the easiest way to get around the issue of a dynamic table. As a result, we will be using this method to create the link table. I will update the file paths to be generic and add the necessary files to the main directory so that this can be run elsewhere. 

An additional issue is that each search and window close takes time. As a result, based on the size of the data I will be working with, I will need to utilize parallel execution along with other methods to reduce the runtime. 

In [5]:
#driver = webdriver.Firefox() #executable_path=r'your\path\geckodriver.exe'
#driver.get(url)

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#from selenium.webdriver.firefox.options import Options

start_time = time.time()


#options = Options()
#options.binary_location = './Mozilla Firefox/firefox.exe' #r'C:\Program Files\Mozilla Firefox\firefox.exe'
#driver = webdriver.Firefox(executable_path='./geckodriver.exe', options=options)
driver = webdriver.Chrome()
driver.get(url)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#dijit_layout_ContentPane_0 > div')))

html = driver.page_source

driver.close()

print("My program took", time.time()-start_time, "to run")

My program took 8.204723119735718 to run


In [6]:
start_time = time.time()

#req = requests.get(html, headers)
sample = BeautifulSoup(html, 'html.parser') #req.content

print("My program took", time.time()-start_time, "to run")

My program took 0.08078145980834961 to run


In [7]:
# navigating to dynamic search query table and obtaining a list of the 'Name' element/col of each row
query_table = sample.find_all(name='div', class_ = 'table-responsive')
query_list = query_table[0].find_all(name='td', style = 'text-align: left;')
len(query_list)

3

In [8]:
# creating mock link table

vs_name = ['Campaign for Working Families']
vs_id = ['1145']
names = []
ids = []


for row in query_list:
    names.append(row.string)
    ids.append(row['tokenvalue'])
    print(row.string + ': ' + row['tokenvalue'])
    
temp = np.array(vs_name)
vs_name = np.repeat(temp,len(query_list))
temp = np.array(vs_id)
vs_id = np.repeat(temp,len(query_list))

    
dict = {'vs_name': vs_name, 'vs_id': vs_id, 'org_name': names, 'ID': ids}

df = pd.DataFrame(dict)
df

CAMPAIGN FOR WORKING FAMILIES: 471
CAMPAIGN FOR WORKING FAMILIES PAC CWF: 46277079
NEW HAMPSHIRE CAMPAIGN FOR WORKING FAMILIES: 16629025


,vs_name,vs_id,org_name,ID
0,Campaign for Working Families,1145,CAMPAIGN FOR WORKING FAMILIES,471
1,Campaign for Working Families,1145,CAMPAIGN FOR WORKING FAMILIES PAC CWF,46277079
2,Campaign for Working Families,1145,NEW HAMPSHIRE CAMPAIGN FOR WORKING FAMILIES,16629025


#### New Export: Write rows to csv

This new information extraction/organization method is meant to be utilized with the parallelized scraping later

In [9]:
rows = []
vs_name = 'Campaign for Working Families'
vs_id = '1145'
for query in query_list:
    row = [vs_name, vs_id, query.string, query['tokenvalue']]
    rows.append(row)

In [10]:
rows[0]

['Campaign for Working Families',
 '1145',
 'CAMPAIGN FOR WORKING FAMILIES',
 '471']

In [11]:
import csv
file = open('practice.csv', 'w')
writer = csv.writer(file)
writer.writerow(['vs_name', 'vs_id', 'ftm_name','ftm_id'])
for row in rows:
    writer.writerow(row)
file.close()

### Search & Match Automation

#### Query Creation

For both tasks we assume that we are given a list of strings where each string is an organization name that is separated by spaces (if longer than one word) and is of unknown capitalization. 

In [12]:
in_string = 'https://justfacts.votesmart.org/interest-group/369/eagle-forum-of-alabama' # sample string from list
vs_id = in_string.split("/")[-2]
org_name = in_string.split("/")[-1]
search_string = "+".join(org_name.title().split("-"))
query = 'https://www.followthemoney.org/search-results/SearchForm?Search=' + search_string
print(query)

https://www.followthemoney.org/search-results/SearchForm?Search=Eagle+Forum+Of+Alabama


#### Organization Matching

**Plan for matching:**

0. Check how many queries return more than one organization match
1. Start with strict matching i.e. checking if strings are exactly the same and determine how many there are missing
2. Depending on the results of (1), look into near matches (utilize split and check how many words match in the list and/or create a specific threshold e.g. x number of words match)
3. Based on results of (2), devise a strategy to fill in remaining orgs if necessary

### Importing Org Names

In [13]:
vs_names = pd.read_csv('SIG_state.csv')

In [14]:
vs_names.shape

(1617, 1)

In [15]:
vs_names.columns

Index(['state;sig_url'], dtype='object')

In [16]:
names_list = vs_names['state;sig_url'].values.tolist()

In [17]:
len(names_list)

1617

In [18]:
type(names_list)

list

In [19]:
names_list[0]

'AL;https://justfacts.votesmart.org/interest-group/369/eagle-forum-of-alabama'

### Parallelizing Final Scraping

https://www.guru99.com/sessions-parallel-run-and-dependency-in-selenium.html
https://gist.github.com/wooddar/df4c89f381fa20ce819e94782dc5bc04

### Test Runs

#### Test Run #1

In [20]:
import csv
file = open('draft_table.csv', 'w')
writer = csv.writer(file)
writer.writerow(['vs_link','vs_name', 'vs_id', 'ftm_name','ftm_id'])
file.close()

In [21]:
from parallelization import parallel
from vs_ltm_linktable import scraping

parallel(scraping,names_list)

Exception in thread Thread-9:
Traceback (most recent call last):
  File "C:\Users\matth\anaconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\matth\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\matth\Documents\Coding\RA Fouirnaies\vs_ltm_linktable\parallelization.py", line 45, in selenium_queue_listener
    selenium_task(worker, current_data)
  File "C:\Users\matth\Documents\Coding\RA Fouirnaies\vs_ltm_linktable\parallelization.py", line 26, in selenium_task
    function(worker, data)
  File "C:\Users\matth\Documents\Coding\RA Fouirnaies\vs_ltm_linktable\vs_ltm_linktable.py", line 73, in scraping
    worker.get(query)
  File "C:\Users\matth\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 333, in get
    self.execute(Command.GET, {'url': url})
  File "C:\Users\matth\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 321, in execute
    self

This parallel operation took 4186.534124135971 to run


In [22]:
'''
start_time = time.time()
from multiprocessing import Queue, cpu_count
from threading import Thread
from selenium import webdriver
from time import sleep
from numpy.random import randint
import logging
from vs_ltm_linktable import scraping


logger = logging.getLogger(__name__)

# Some example data to pass the the selenium processes, this will just cause a sleep of time i
# This data can be a list of any datatype that can be pickled

start = 0
end = len(names_list)
selenium_data = names_list[start:end]
selenium_data.append("STOP")

# Create the two queues to hold the data and the IDs for the selenium workers
selenium_data_queue = Queue()
worker_queue = Queue()

# Create Selenium processes and assign them a worker ID
# This ID is what needs to be put on the queue as Selenium workers cannot be pickled
# By default, make one selenium process per cpu core with cpu_count
# TODO: Change the worker creation code to be your webworker of choice e.g. PhantomJS
worker_ids = list(range(4)) #changed number of worker_ids from cpu_count() to 4
selenium_workers = {i: webdriver.Chrome() for i in worker_ids}
for worker_id in worker_ids:
    worker_queue.put(worker_id)


def selenium_task(worker, data):
    """
    This is a demonstration selenium function that takes a worker and data and then does something with the worker and
    data.

    TODO: change the below code to be whatever it is you want your worker to do e.g. scrape webpages or run browser tests

    :param worker: A selenium web worker NOT a worker ID
    :type worker: webdriver.XXX
    :param data: Any data for your selenium function (must be pickleable)
    :rtype: None
    """
    scraping(worker, data)


def selenium_queue_listener(data_queue, worker_queue):
    """
    Monitor a data queue and assign new pieces of data to any available web workers to action

    :param data_queue: The python FIFO queue containing the data to run on the web worker
    :type data_queue: Queue
    :param worker_queue: The queue that holds the IDs of any idle workers
    :type worker_queue: Queue
    :rtype: None
    """
    logger.info("Selenium func worker started")
    while True:
        current_data = data_queue.get()
        if current_data == 'STOP':
            # If a stop is encountered then kill the current worker and put the stop back onto the queue
            # to poison other workers listening on the queue
            logger.warning("STOP encountered, killing worker thread")
            data_queue.put(current_data)
            break
        else:
            logger.info(f"Got the item {current_data} on the data queue")
        # Get the ID of any currently free workers from the worker queue
        worker_id = worker_queue.get()
        worker = selenium_workers[worker_id]
        # Assign current worker and current data to your selenium function
        selenium_task(worker, current_data)
        # Put the worker back into the worker queue as  it has completed it's task
        worker_queue.put(worker_id)
    return


# Create one new queue listener thread per selenium worker and start them
logger.info("Starting selenium background processes")
selenium_processes = [Thread(target=selenium_queue_listener,
                             args=(selenium_data_queue, worker_queue)) for _ in worker_ids]
for p in selenium_processes:
    p.daemon = True
    p.start()

# Add each item of data to the data queue, this could be done over time so long as the selenium queue listening
# processes are still running
logger.info("Adding data to data queue")
for d in selenium_data:
    selenium_data_queue.put(d)

# Wait for all selenium queue listening processes to complete, this happens when the queue listener returns
logger.info("Waiting for Queue listener threads to complete")
for p in selenium_processes:
    p.join()

# Quit all the web workers elegantly in the background
logger.info("Tearing down web workers")
for b in selenium_workers.values():
    b.quit()

print("Scraping", end-start, "organizations took", time.time()-start_time, "to run")
'''

'\nstart_time = time.time()\nfrom multiprocessing import Queue, cpu_count\nfrom threading import Thread\nfrom selenium import webdriver\nfrom time import sleep\nfrom numpy.random import randint\nimport logging\nfrom vs_ltm_linktable import scraping\n\n\nlogger = logging.getLogger(__name__)\n\n# Some example data to pass the the selenium processes, this will just cause a sleep of time i\n# This data can be a list of any datatype that can be pickled\n\nstart = 0\nend = len(names_list)\nselenium_data = names_list[start:end]\nselenium_data.append("STOP")\n\n# Create the two queues to hold the data and the IDs for the selenium workers\nselenium_data_queue = Queue()\nworker_queue = Queue()\n\n# Create Selenium processes and assign them a worker ID\n# This ID is what needs to be put on the queue as Selenium workers cannot be pickled\n# By default, make one selenium process per cpu core with cpu_count\n# TODO: Change the worker creation code to be your webworker of choice e.g. PhantomJS\nworke

### Data Validation

#### Checking Organizations

Check the number of organizations sucessfully scraped and gaps in matching, etc.

In [4]:
temp = pd.read_csv("temp_draft_table.csv")

In [5]:
len(temp['vs_name'].unique())

1582

In [6]:
temp.shape

(3173, 5)

In [7]:
temp.dtypes

vs_link     object
vs_name     object
vs_id        int64
ftm_name    object
ftm_id      object
dtype: object

In [17]:
temp[temp['ftm_name']=='TimeoutException'].shape[0] # number of scrapes that ran into timeoutexception

641

In [19]:
len(temp[temp['ftm_name']=='TimeoutException']['vs_link'].unique()) # numbers don't match but in theory should because
# when an error occurs, the function should output a row with timeoutexception filled in for values once and then exit and move on

540

In [20]:
len(temp[temp['ftm_name']=='ResultsEmpty']['vs_link'].unique()) # number of scrapes where a table showed up but there were no matches

0

In [10]:
rerun = temp[temp['ftm_name']=='TimeoutException']['vs_link'].tolist()
type(rerun)

list

In [ ]:
from parallelization import parallel
from vs_ltm_linktable import scraping

parallel(scraping,rerun)

In [ ]:
temp.head()